#GigaChat с использованием Retrieval Augmented Generation(RAG) и self-asking (через промпты) (первая версия, вопросы задаются до получения основного ответа)

##Установка необходимых библиотек

In [2]:
!pip install gigachain -q
!pip install -U gigachain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.1/341.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.3 MB/s eta 0:00:00


In [3]:
#!pip install --user faiss-cpu -q
!pip install sentence-transformers -q
!pip install faiss-cpu
!pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.3 MB/s eta 0:00:00


##Инициализируем модель

In [4]:
from getpass import getpass
from langchain.chat_models.gigachat import GigaChat

In [5]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

Введите API ключ от GigaChat··········


In [6]:
# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

##Подготовка документа

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

In [8]:
# Загружаем и разбиваем документ
loader = PyPDFLoader("/content/drive/MyDrive/GigaChat_doc/Приказ ФСТЭК России от 25 декабря 2017 г. N 239.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_doc = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 37


##Создание базы данных эмбеддингов

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [10]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru",
        model_kwargs={"device": "cpu"}
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_doc, hf_embeddings_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

##Создание цепочки

Создадим цепочку, которая предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель, ретривер (база данных) и доп информация на основе вопроса.

###Пример ответа модели без дополнительных вопросов

In [11]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import re

In [12]:
# Создаём простой шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Задаем ретривер
retriever=db.as_retriever()

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
messege = chain.invoke("В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?")
print(messege)

Если установка наложенных средств защиты информации (СрЗИ) невозможна на объекте критической информационной инфраструктуры (КИИ), то необходимо выполнить следующие действия:

1. Определить базовый набор мер по обеспечению безопасности значимого объекта.
2. Выбрать меры по обеспечению безопасности значимых объектов для их реализации.
3. Разработать рабочую (эксплуатационную) документацию на значимый объект.
4. Описать архитектуру подсистемы безопасности значимого объекта.
5. Определить порядок и параметры настройки программных и программно-аппаратных средств, включая средства защиты информации.

При этом следует учитывать, что выбор мер по обеспечению безопасности значимых объектов должен быть основан на категории значимости объекта КИИ и перечне нормативных правовых актов, методических документов и национальных стандартов, которым должен соответствовать значимый объект.


###Пример ответа модели с дополнительными вопросами

In [14]:
# Создаём шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Задаем ретривер
retriever = db.as_retriever()

# Функция для генерации дополнительных вопросов
def generate_subquestions(question):
    subquestion_answer = chain.invoke(f"Сгенерируй два уточняющих вопроса по данному вопросу: {question} " +
                                      "Каждый вопрос начинай с соответствующей цифры")
    subquestions = re.split(r'\d+.', subquestion_answer)
    subquestions = subquestions[1:]
    subquestions_and_answers = ""
    count = 0
    for subquestion in subquestions:
        subquestion = subquestion.strip().replace("/n", "")
        count += 1
        subquestion_answer = chain.invoke(subquestion)  # Вызываем chain для отдельного вопроса
        subquestions_and_answers += f"Subquestion {count}: {subquestion}\nAnswer: {subquestion_answer}\n\n"
    return subquestions_and_answers

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [18]:
question = "В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?"
subquestions_and_answers = generate_subquestions(question)
full_request = f"{question}\n\n{subquestions_and_answers}"

answer = chain.invoke(f"Ответь на основной вопрос используя ответы на дополнительные вопросы. Основной вопрос: {question} Дополнительные впоросы: {subquestions_and_answers}")
print(answer)

Для котельной, входящей в состав ЗОКИИ, могут быть использованы следующие меры по обеспечению безопасности: определение и обоснование организационных и технических мер, подлежащих реализации в рамках подсистемы безопасности значимого объекта; определение видов и типов средств защиты информации, обеспечивающих реализацию технических мер по обеспечению безопасности значимого объекта; выбор средств защиты информации и (или) их разработка с учетом категории значимости значимого объекта, совместимости с программными и программно-аппаратными средствами, выполняемых функций безопасности и ограничений на эксплуатацию; разработка архитектуры подсистемы безопасности значимого объекта, включающей состав, места установки, взаимосвязи средств защиты информации; определение требований к параметрам настройки программных и программно-аппаратных средств, включая средства защиты информации, обеспечивающие реализацию мер по обеспечению безопасности, блокирование угроз безопасности информации.
